In [1]:
#set ulimit to 10k before running notebook to accomodate num_df file descriptors. ~200 workers * 25 fd/worker = 5k
!ulimit -n
#!ulimit -Sn 10000

10000


In [2]:
import pandas as pd
import numpy as np
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split

In [3]:
from ipywidgets import IntProgress
IntProgress(10,max=100)

IntProgress(value=10)

In [4]:
import tpot
tpot.__version__

'0.9.5'

In [5]:
# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=1)
# client

In [6]:
from dask_jobqueue import LSFCluster
from dask.distributed import Client, progress
from timeit import default_timer as timer
import dask.bag as db
import dask.dataframe as dd
import gc
# try:
#     gc.enable()
#     del cluster
#     gc.collect(cluster)
# except:
#     pass
# cluster = LSFCluster(queue='corradin_long',
#                      cores= 2,
#                      #ncpus=2,
#                      walltime='1000000:00',
#                      memory='15GB',
#                      death_timeout=600,
#                      job_extra=['-o /dev/null', '-g /dask-workers', '-e error_log_worker' ]#, '']
#                      )
# cluster.adapt(minimum= 200, maximum=200)
# #cluster.scale(250)
# print(cluster.job_script())
# cluster.dashboard_link

In [7]:
data = pd.read_csv("ML_Features_March2018.csv")
data.head()

,Pat_Enc_CSN_ID,ADT_Arrival_Time,EDDepartureDt,First_IP_BedRequestDt,LOS,timeToRoom,Age,ArrivalMethod,First_CareArea,Last_CareArea,...,PendingMed,PendingSurg,PendingICU,PendingAll_RequestTime,PendingMed_RequestTime,PendingSurg_RequestTime,PendingICU_RequestTime,ShareEDMedBedRequests_RequestTime,ShareEDSurgBedRequests_RequestTime,ShareEDICUBedRequests_RequestTime
0,3226269117,11/19/18 21:29,11/20/18 15:27,11/20/18 3:21,1078.0,45.0,80,Car,Evalulation,Urgent,...,123,15,1,346,120,15,1,0.200000,0.285714,0.500000
1,3226274255,11/20/18 4:15,11/20/18 15:08,11/20/18 5:35,653.0,7.0,71,Ambulance,Acute,Acute,...,126,16,1,352,123,16,1,0.216561,0.238095,0.666667
2,3226636912,11/23/18 9:43,11/23/18 13:42,NaN,239.0,19.0,59,Public Transportation,Evalulation,CDU,...,0,0,0,344,119,14,5,NaN,NaN,NaN
3,3226646686,11/23/18 10:56,11/23/18 21:07,11/23/18 13:34,611.0,8.0,55,Ambulance,Acute,Acute,...,131,18,6,390,149,21,6,0.050955,0.000000,0.000000
4,3226655334,11/23/18 11:59,11/23/18 15:06,11/23/18 14:17,187.0,3.0,6,Car,Pedi,Pedi,...,126,18,5,375,140,20,6,0.072848,0.000000,0.000000


In [8]:
string_cols = [x for x in data.columns if data[x].dtypes == "object"]
string_cols
#string_cols = [data[x].astype("category", copy=False) for x in string_cols]

['ADT_Arrival_Time',
 'EDDepartureDt',
 'First_IP_BedRequestDt',
 'ArrivalMethod',
 'First_CareArea',
 'Last_CareArea',
 'ActualLocationAfterED',
 'First_EDAttending',
 'ChiefComplaint',
 'DispoGrp']

In [9]:
float_cols = [x for x in data.columns if data[x].dtypes == "float64"]
float_cols

['LOS',
 'timeToRoom',
 'MedBedsUtil',
 'SurgBedsUtil',
 'ICUBedsUtil',
 'OtherBedsUtil',
 'MedBedsAvailable_RequestTime',
 'SurgBedsAvailable_RequestTime',
 'ICUBedsAvailable_RequestTime',
 'OtherBedsAvailable_RequestTime',
 'MedBedsUtil_RequestTime',
 'SurgBedsUtil_RequestTime',
 'ICUBedsUtil_RequestTime',
 'OtherBedsUtil_RequestTime',
 'EDMedBedRequests_RequestTime',
 'EDSurgBedRequests_RequestTime',
 'EDICUBedRequests_RequestTime',
 'ShareEDMedBedRequests_RequestTime',
 'ShareEDSurgBedRequests_RequestTime',
 'ShareEDICUBedRequests_RequestTime']

In [10]:
# for x in string_cols:
#     data[x].astype("category", copy=False)
data[string_cols] = data[string_cols].astype("category")
data[float_cols] = data[float_cols].astype(np.float64)
data.dtypes

Pat_Enc_CSN_ID                           int64
ADT_Arrival_Time                      category
EDDepartureDt                         category
First_IP_BedRequestDt                 category
LOS                                    float64
timeToRoom                             float64
Age                                      int64
ArrivalMethod                         category
First_CareArea                        category
Last_CareArea                         category
DoW                                      int64
ToD                                      int64
ActualLocationAfterED                 category
First_EDAttending                     category
ChiefComplaint                        category
AreaChange                               int64
DoY                                      int64
EDCensus                                 int64
numUrgAcuteBoarders                      int64
OBSFlg                                   int64
DispoGrp                              category
NumUrgAcuteDe

In [11]:
data.isnull().sum()

Pat_Enc_CSN_ID                            0
ADT_Arrival_Time                          0
EDDepartureDt                            90
First_IP_BedRequestDt                 16492
LOS                                      90
timeToRoom                              201
Age                                       0
ArrivalMethod                            18
First_CareArea                          175
Last_CareArea                           296
DoW                                       0
ToD                                       0
ActualLocationAfterED                 16686
First_EDAttending                       280
ChiefComplaint                         2066
AreaChange                                0
DoY                                       0
EDCensus                                  0
numUrgAcuteBoarders                       0
OBSFlg                                    0
DispoGrp                                 90
NumUrgAcuteDepartures                     0
EDOBSCensus                     

In [12]:
columns_pre_filter = set(data.columns)
columns_post_filter = set(data.dropna(axis = 1, thresh=data.shape[0] * 0.9).columns)
columns_filtered = columns_pre_filter - columns_post_filter
columns_filtered

{'ActualLocationAfterED',
 'EDICUBedRequests_RequestTime',
 'EDMedBedRequests_RequestTime',
 'EDSurgBedRequests_RequestTime',
 'First_IP_BedRequestDt',
 'ICUBedsAvailable_RequestTime',
 'ICUBedsUtil_RequestTime',
 'MedBedsAvailable_RequestTime',
 'MedBedsUtil_RequestTime',
 'OtherBedsAvailable_RequestTime',
 'OtherBedsUtil_RequestTime',
 'ShareEDICUBedRequests_RequestTime',
 'ShareEDMedBedRequests_RequestTime',
 'ShareEDSurgBedRequests_RequestTime',
 'SurgBedsAvailable_RequestTime',
 'SurgBedsUtil_RequestTime'}

In [13]:
filtered_data = data[list(columns_post_filter)]
filtered_data.shape

(22258, 60)

In [14]:
filtered_data = filtered_data.drop(columns=["Pat_Enc_CSN_ID", "ADT_Arrival_Time", "EDDepartureDt"])#, "First_IP_BedRequestDt"])
filtered_data.head()


,EDOBSCensus,NumCT,LOS,DoY,CDUCensus,OtF,ChiefComplaint,OtP,NumUS,ICUBedsAvailable,...,PendingSurg,APSCensus,NumScans,OtherBedsAvailable,Last_CareArea,PediCensus,UrgentAndAcuteWaitingForAdmit,ICUBedsUtil,DoW,NumSurgConsults
0,25,0,1078.0,-323,31,88,WEIGHT LOSS,0,0,8,...,15,5,1,208,Urgent,12,24,0.098765,2,0
1,28,0,653.0,-324,23,0,DIVERTICULITIS,0,0,8,...,16,6,0,203,Acute,0,42,0.101266,3,1
2,17,0,239.0,-327,7,0,MIGRAINE,0,0,8,...,0,4,0,228,CDU,1,12,0.100000,6,0
3,16,0,611.0,-327,10,21,CHEST PAIN,17,0,6,...,18,5,1,211,Acute,1,10,0.075949,6,0
4,16,0,187.0,-327,15,223,ABDOMINAL PAIN,148,1,6,...,18,5,1,207,Pedi,2,10,0.075949,6,1


In [15]:
dummied = filtered_data.copy()
dummied.shape

(22258, 57)

In [16]:
dummied = dummied[~dummied.isna().any(axis=1)]
dummied.shape

(19889, 57)

In [17]:
dummied = pd.get_dummies(dummied, dummy_na=True)
dummied.head()

,EDOBSCensus,NumCT,LOS,DoY,CDUCensus,OtF,OtP,NumUS,ICUBedsAvailable,OtherBedsUtil,...,ArrivalMethod_nan,Last_CareArea_APS,Last_CareArea_Acute,Last_CareArea_CDU,Last_CareArea_Evalulation,Last_CareArea_Fast Track,Last_CareArea_Pedi,Last_CareArea_Surge,Last_CareArea_Urgent,Last_CareArea_nan
0,25,0,1078.0,-323,31,88,0,0,8,0.371429,...,0,0,0,0,0,0,0,0,1,0
1,28,0,653.0,-324,23,0,0,0,8,0.362500,...,0,0,1,0,0,0,0,0,0,0
2,17,0,239.0,-327,7,0,0,0,8,0.402827,...,0,0,0,1,0,0,0,0,0,0
3,16,0,611.0,-327,10,21,17,0,6,0.370826,...,0,0,1,0,0,0,0,0,0,0
4,16,0,187.0,-327,15,223,148,1,6,0.364437,...,0,0,0,0,0,0,1,0,0,0


In [18]:
dummied.dtypes

EDOBSCensus                              int64
NumCT                                    int64
LOS                                    float64
DoY                                      int64
CDUCensus                                int64
OtF                                      int64
OtP                                      int64
NumUS                                    int64
ICUBedsAvailable                         int64
OtherBedsUtil                          float64
ToD                                      int64
SurgBedsAvailable                        int64
EDCensus                                 int64
NumUrgAcuteDepartures                    int64
NumXR                                    int64
NumMR                                    int64
NumLabsOrdered                           int64
PendingICU                               int64
SurgBedsUtil                           float64
UrgentCensus                             int64
PendingMed_RequestTime                   int64
AreaChange   

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    dummied.drop(columns="LOS").values,
    dummied.LOS.values,
    train_size=0.8,
    test_size=0.2,
)

***
## Attempt 1: using 2 cores/worker, 200 workers, n_jobs = 100

In [5]:
from dask_jobqueue import LSFCluster
from dask.distributed import Client, progress
from timeit import default_timer as timer
import dask.bag as db
import dask.dataframe as dd
import gc
try:
    gc.enable()
    del cluster
    gc.collect(cluster)
except:
    pass
cluster = LSFCluster(queue='corradin_long',
                     cores= 2,
                     #ncpus=2,
                     walltime='100000:00',
                     memory='15GB',
                     death_timeout=600,
                     job_extra=['-o /dev/null', '-g /dask-workers', '-e error_log_worker' ]#, '']
                     )
cluster.adapt(minimum= 200, maximum=200)
#cluster.scale(250)
print(cluster.job_script())
cluster.dashboard_link
client = Client(cluster)
client


#!/bin/bash

#BSUB -J dask-worker
#BSUB -q corradin_long
#BSUB -n 2
#BSUB -R "span[hosts=1]"
#BSUB -M 15000
#BSUB -W 100000:00
#BSUB -o /dev/null
#BSUB -g /dask-workers
#BSUB -e /dev/null
JOB_ID=${LSB_JOBID%.*}



/home/anhoang/.conda/envs/dask-test/bin/python -m distributed.cli.dask_worker tcp://172.18.10.1:35037 --nthreads 2 --memory-limit 15.00GB --name dask-worker--${JOB_ID}-- --death-timeout 600



'http://172.18.10.1:8787/status'

In [27]:
# scale up: Increase the TPOT parameters like population_size, generations
tp = TPOTRegressor(
#     generations=2,
    population_size=1000,
#     cv=2,
    n_jobs=100,
    warm_start=True,
    random_state=0,
    verbosity=3,
    periodic_checkpoint_folder = "./tpot_checkpoints",
    memory = "./memory",
    use_dask=True,
    #config_dict = 'TPOT sparse'
)

In [ ]:
client

Client Scheduler: tcp://172.18.10.1:35037 Dashboard: http://172.18.10.1:8787/status,Cluster Workers: 200 Cores: 400 Memory: 3.00 TB


In [ ]:
tp.fit(X_train, y_train)

29 operators have been imported by TPOT.


Worker tcp://172.18.113.7:41473 restart in Job 7063541. This can be due to memory issue.
Worker tcp://172.18.113.7:37111 restart in Job 7063643. This can be due to memory issue.
Worker tcp://172.18.113.10:33667 restart in Job 7063655. This can be due to memory issue.
Worker tcp://172.18.113.11:41213 restart in Job 7063659. This can be due to memory issue.
Worker tcp://172.18.113.8:37241 restart in Job 7063673. This can be due to memory issue.
Worker tcp://172.18.113.9:35103 restart in Job 7063671. This can be due to memory issue.
Worker tcp://172.18.113.6:32999 restart in Job 7063592. This can be due to memory issue.
Worker tcp://172.18.113.6:32861 restart in Job 7063661. This can be due to memory issue.
Worker tcp://172.18.113.6:32861 restart in Job 7063661. This can be due to memory issue.
Worker tcp://172.18.113.6:32999 restart in Job 7063592. This can be due to memory issue.
Worker tcp://172.18.113.1:40805 restart in Job 7063675. This can be due to memory issue.
Worker tcp://172.18

In [ ]:
tp.score(X_train, y_train)

***
## Attempt 2: using 4 cores/worker, 100 workers, increased memory to 30GB each, n_jobs = 100

In [20]:
from dask_jobqueue import LSFCluster
from dask.distributed import Client, progress
from timeit import default_timer as timer
import dask.bag as db
import dask.dataframe as dd
import gc
try:
    gc.enable()
    del cluster
    del client
    gc.collect(cluster)
    gc.collect(client)
except:
    pass


In [21]:
cluster = LSFCluster(queue='corradin_long',
                     cores= 4,
                     #ncpus=2,
                     walltime='100000:00',
                     memory='15GB',
                     death_timeout=600,
                     job_extra=['-o /dev/null', '-g /dask-workers', '-e error_log_worker' ]#, '']
                     )
cluster.adapt(minimum= 100, maximum=100)
#cluster.scale(250)
print(cluster.job_script())
cluster.dashboard_link
client = Client(cluster)


#!/bin/bash

#BSUB -J dask-worker
#BSUB -q corradin_long
#BSUB -n 4
#BSUB -R "span[hosts=1]"
#BSUB -M 15000
#BSUB -W 100000:00
#BSUB -o /dev/null
#BSUB -g /dask-workers
#BSUB -e error_log_worker
JOB_ID=${LSB_JOBID%.*}



/home/anhoang/.conda/envs/dask-test/bin/python -m distributed.cli.dask_worker tcp://172.18.10.1:36181 --nthreads 4 --memory-limit 15.00GB --name dask-worker--${JOB_ID}-- --death-timeout 600



In [22]:
# scale up: Increase the TPOT parameters like population_size, generations
tp = TPOTRegressor(
#     generations=2,
    population_size=1000,
#     cv=2,
    n_jobs=50,
    warm_start=True,
    random_state=0,
    verbosity=3,
    periodic_checkpoint_folder = "./tpot_checkpoints",
    memory = "./memory",
    use_dask=True,
    #config_dict = 'TPOT sparse'
)

In [34]:
#executed this cell for a few times until all workers arrived
client

Client Scheduler: tcp://172.18.10.1:36181 Dashboard: http://172.18.10.1:8787/status,Cluster Workers: 100 Cores: 400 Memory: 1.50 TB


In [33]:
tp.fit(X_train, y_train)

29 operators have been imported by TPOT.


Worker tcp://172.18.113.9:35275 restart in Job 7066784. This can be due to memory issue.
Worker tcp://172.18.113.9:33279 restart in Job 7066773. This can be due to memory issue.
Worker tcp://172.18.113.5:40349 restart in Job 7066824. This can be due to memory issue.
Worker tcp://172.18.113.4:41579 restart in Job 7066806. This can be due to memory issue.
Worker tcp://172.18.113.3:44587 restart in Job 7066847. This can be due to memory issue.
Worker tcp://172.18.113.11:37609 restart in Job 7066800. This can be due to memory issue.
Worker tcp://172.18.113.8:42559 restart in Job 7066752. This can be due to memory issue.
Worker tcp://172.18.113.1:42815 restart in Job 7066791. This can be due to memory issue.
Worker tcp://172.18.113.9:40021 restart in Job 7066808. This can be due to memory issue.
Worker tcp://172.18.113.9:34691 restart in Job 7066831. This can be due to memory issue.
Worker tcp://172.18.113.9:33993 restart in Job 7066750. This can be due to memory issue.
Worker tcp://172.18.

Worker tcp://172.18.113.4:33597 restart in Job 7066872. This can be due to memory issue.
distributed.utils - ERROR - Worker already exists tcp://172.18.113.4:33597
Traceback (most recent call last):
  File "/home/anhoang/.conda/envs/dask-test/lib/python3.6/site-packages/distributed/utils.py", line 649, in log_errors
    yield
  File "/home/anhoang/.conda/envs/dask-test/lib/python3.6/site-packages/distributed/scheduler.py", line 1281, in add_worker
    raise ValueError("Worker already exists %s" % address)
ValueError: Worker already exists tcp://172.18.113.4:33597
distributed.core - ERROR - Worker already exists tcp://172.18.113.4:33597
Traceback (most recent call last):
  File "/home/anhoang/.conda/envs/dask-test/lib/python3.6/site-packages/distributed/core.py", line 346, in handle_comm
    result = yield result
  File "/home/anhoang/.conda/envs/dask-test/lib/python3.6/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/anhoang/.conda/envs/dask-te

RuntimeError: A pipeline has not yet been optimized. Please call fit() first.

In [ ]:
tp.score(X_train, y_train)

In [4]:
!ulimit -n
#!ulimit -Sn 10000

1024
